## Illustration of predict_proba

based on: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB

In [1]:
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [2]:
clf = GaussianNB()
clf.fit(X, Y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [3]:
clf.predict([[-0.8, -1]])

array([1])

In [4]:
clf.predict_proba([[-0.8, -1]])

array([[9.99999949e-01, 5.05653254e-08]])

## Save model

In [5]:
from joblib import dump, load
dump(clf, './model.joblib')

['./model.joblib']

## Upload model

In [6]:
from zipfile import ZipFile
from io import BytesIO
import requests
from requests.auth import HTTPBasicAuth
def upload_model(model_name, description, upload_url, api_key, api_secret):
    
    zipfile_name = 'model.zip'
    # Create a zip
    with ZipFile('model.zip', 'w') as zipfile:
       # Add multiple files to the zip
       zipfile.write('model.joblib')
        
    with open('model.zip', 'rb') as f:
        upload_file = BytesIO(f.read())
        
    # upload to waylay
    resp = requests.post(upload_url,
                         files={"file": ('model.zip', upload_file)},
                         data={"name": model_name, "framework": "sklearn", "description": description},
                         auth=HTTPBasicAuth(api_key, api_secret))

    return resp.json()

In [8]:
# https://docs.waylay.io/api/rest/#authentication
api_key = 'your api key'
api_secret = 'your api secret'
byom_url = 'https://byoml.waylay.io/models'
model_name = 'probabilities'
description = 'This model also support probabilities'
print(
    upload_model(
        model_name,
        description,
        byom_url,  
        api_key, 
        api_secret
    )
)

{'message': 'Model successfully uploaded'}


## Check if model and uploaded model act the same

In [9]:
features = [[-0.8, -1]]

In [10]:
clf.predict(features)

array([1])

In [11]:
response = requests.post(byom_url + '/' + model_name + '/predict',
                         json = {"instances": features},
                         auth=HTTPBasicAuth(api_key, api_secret)).json()
response

{'predictions': [1]}

In [16]:
clf.predict_proba(features)

array([[9.99999949e-01, 5.05653254e-08]])

In [17]:
response = requests.post(byom_url + '/' + model_name + '/predict',
                         json = {"instances": features, 'probabilities': True},
                         auth=HTTPBasicAuth(api_key, api_secret)).json()
response

{'predictions': [[0.9999999494346747, 5.056532536795868e-08]]}

## Delete model

In [ ]:
requests.delete(byom_url + '/' + model_name,
                auth=HTTPBasicAuth(api_key, api_secret)).json()